RDD 创建需要依赖于 SparkContext——它是用于连接 Spark 集群创建 RDD 以及广播集群中变量。

1. [SparkContext 创建 Spark 应用](#SparkContext创建Spark应用)
2. [SparkSession 创建 Spark 应用](#SparkSession创建Spark应用)
3. [查看Spark应用配置信息](#查看Spark应用配置信息)


### SparkContext创建Spark应用
直接通过 SparkContext 创建，可以使用 SparkConf 对 Spark 应用进行设置， SparkConf 会默认加载 `spark.*` 的 Java 系统属性，也可以通过传入参数修改相关属性。进行单元测试时，可以使用 `loadDefaults=False` 参数来使用系统属性配置以保持测试一致。此外 SparkConf 还有其他方法用于设置，例如 `conf.setMaster("local").setAppName("App")`。

In [1]:
# 通过 SparkContext 创建
from pyspark import SparkContext, SparkConf

# 通过 SparkConf 进行设相关属性，需要以 key-value 形式来进行设置
config = SparkConf() \
        .set("Name", "ContextTest") \
        .setMaster("local") \
        .setAppName("First App")


# 创建 SparkContext
sc = SparkContext(conf=config)

### 使用parallelize创建RDD
[parallelize](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.parallelize) 是用于创建 RDD 的便捷方法

In [8]:
test = sc.parallelize(range(0, 10, 2), 3)

In [12]:
test.map(lambda a: a**2).collect()

[0, 4, 16, 36, 64]

In [19]:
def f(x): print(x)
sc.parallelize([1, 2, 3, 4, 5]).foreach(f)

In [17]:
test.foreach(lambda x: x).collect()

AttributeError: 'NoneType' object has no attribute 'collect'

In [9]:
type(test)

pyspark.rdd.PipelinedRDD

### SparkSession创建Spark应用
SparkSession 是 应用中 DataFrame 和 SQL 的入口，可以用于创建 DataFrame、注册 DataFrame 为 Table（可以对 Table 使用 SQl 语句），也可以用于创建 RDD。SparkSession 创建也可以通过以下两种方式创建：
1. 通过 builder 创建 对 Spark 应用的设置需要通过 SparkSession 下的相关方法和类（builder 可以用于配置后续相关设置，它是 SparkSession 类嵌套类的实例化）或者相关参数完成。注意在使用 conf 的配置方式和 SparkContext 的配置相似
2. SparkSession 直接实例化 通过传入 SparkContext 作为参数进行实例化对象:
    ```{python}
    from pyspark import SparkContext, SparkConf
    config = SparkConf() \
            .setMaster("local") \
            .setAppName("First App")

    sc = SparkContext(conf=config)

    from pyspark.sql import SparkSession
    ss = SparkSession(sc)
    ```

In [4]:
# sparksession 和 sparkcontext 的导入模块不一致，这点需要注意
from pyspark.sql import SparkSession

In [6]:
ss = SparkSession.builder \
    .master("local") \
    .config("Name", "SessionTest") \
    .appName("SessionApp") \
    .getOrCreate()

## 查看Spark应用配置信息
获取 SparkContext 配置信息的方式，可以通过 getConf 方法以及其他 get 方法配合使用查看，例如：`sc.getConf().get("spark.master")` 等，需要注意 Spark 应用的配置信息——基本上非配置的独特属性情况下，配置值都是以 `spark.*` 的形式生成的

In [2]:
# 确认所有配置信息
sc.getConf().getAll()

[('spark.master', 'local'),
 ('spark.app.id', 'local-1575948343502'),
 ('spark.rdd.compress', 'True'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.driver.host', 'f6d772bd8388'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.name', 'First App'),
 ('spark.driver.port', '39263'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('Name', 'ContextTest')]

In [3]:
# 指定配置信息
sc.getConf().get("spark.master")

'local'